In [2]:
import robotic as ry
import numpy as np
import time

In [3]:
config = ry.Config()
config.addFile("puzzles/p5-wall-easy.g")

In [4]:
print(config.getFrameNames())

['world', 'floor', 'outwall_right', 'outwall_back', 'outwall_left', 'outwall_front', 'sub-goal1', 'world', 'floor', 'outwall_right', 'outwall_back', 'outwall_left', 'outwall_front', 'sub-goal1', 'camera_top', 'egoJoint', 'ego', 'goal', 'goal_visible', 'wall2_h', 'wall8_h', 'obj', 'goalLarge']


In [5]:
EGO_NAME = "ego"
OBJ_NAME = "obj"
SUB_GOAL_NAME = "sub-goal1"
GOAL_NAME = "goal"
CAMERA_NAME = "camera_top"
FLOOR_NAME = "floor"

GAP = config.getFrame(EGO_NAME).getSize()[1] * 3

In [6]:
config.view() # somehow it breaks if I don't call this before setting the camera
f = config.getFrame('camera_top')
config.view_setCamera(f)

In [ ]:
q_start = config.getFrame(EGO_NAME).getPosition()[:2]
q_obj = config.getFrame(OBJ_NAME).getPosition()[:2]
q_goal = config.getFrame(GOAL_NAME).getPosition()[:2]

In [61]:
komo = ry.KOMO(config, phases=1, slicesPerPhase=1, kOrder=1, enableCollisions=True)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, scale=[1e0])
komo.addObjective([], ry.FS.negDistance, [EGO_NAME, OBJ_NAME], ry.OT.eq, [1e-1])

ret = ry.NLP_Solver(komo.nlp(), verbose=3).solve()
print(ret)

====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1 	lambda:[]
----newton---- initial point f(x):0.018542 alpha:1 beta:1
--newton-- it:   1  |Delta|:  0.0228933  alpha:          1  evals:   2  f(y):   0.017822  ACCEPT
--newton-- stopping: 'absMax(Delta)<options.stopTolerance'
==nlp== it:   0  evals:   2  A(x):   0.017822  f:          0  g:          0  h:   0.133499  |x-x'|:  0.0228933 	stop:DeltaConverge 	x:[0.00645051, -0.02474]
==nlp== it:   1  evals:   2  A(x):   0.124754  mu:5 	lambda:[0, -0.266998]
--newton-- it:   2  |Delta|:   0.124872  alpha:          1  evals:   3  f(y):   0.102484  ACCEPT
--newton-- it:   3  |Delta|:    0.11352  alpha:          1  evals:   4  f(y):   0.084079  ACCEPT
--newton-- it:   4  |Delta|:     0.1032  alpha:          1  evals:   5  f(y):  0.0688681  ACCEPT
--newton-- it:   5  |Delta|:  0.0938185  alpha:          1  evals:   6  f(y):  0.0562972  ACCEPT
--newton-- it:   6  |Delta|:  0.0852896  alpha:          1  eva

In [ ]:
komo = ry.KOMO(config, phases=2, slicesPerPhase=1, kOrder=1, enableCollisions=True)
komo.addQuaternionNorms() # what is this?
komo.addModeSwitch([1.,-1.], ry.SY.stable, [EGO_NAME, OBJ_NAME], True)
komo.addObjective([1.], ry.FS.negDistance, [EGO_NAME, OBJ_NAME], ry.OT.eq, [1e1])
komo.addObjective([2.], ry.FS.positionDiff, [OBJ_NAME, GOAL_NAME], ry.OT.eq, [1e1])

ret = ry.NLP_Solver(komo.nlp(), verbose=3).solve()
print(ret)

In [62]:
config.setJointState(q_start)
config.view()
print(komo.getPath())

[[-0.77439134 -0.95197984]]


In [63]:
for state in komo.getPath():
    config.setJointState(state)
    config.view()
    time.sleep(1)

In [54]:
print(config.getCollisionFree())

False


In [16]:
config.view_close()
del config